In [1]:
from utils import (
        tvmc_compile_and_unpack, 
        relay_soma_conv2d,
        create_demo_file, 
        parse_cli_options,
        load_or_create_random_array
        )
import tvm
import tvm.relay as relay
import tvm.relay.transform as transform
from tvm.driver.tvmc.model import TVMCModel
from tvm.driver.tvmc.compiler import compile_model
from tvm.relay.backend import Executor, Runtime
import numpy as np

# Define a model in relay

Make example without `relay_soma_conv2d`

In [2]:
def create_model():
    input_shape = (1, 3, 16, 16)
    x = relay.var("input", relay.TensorType(input_shape, 'int8'))

    weights_shape = (32, 3, 3, 3)
    special_data = load_or_create_random_array("weights.npy",
                                               weights_shape, np.int8)
    x, params1 = relay_soma_conv2d(x, 'conv1', weights_shape, 
                                   special_data,
                                   np.ones(weights_shape[0]).astype(np.int32), 
                                   act=False, shift_bits=4)
    params = params1

    # create an IR module from the relay expression
    mod = tvm.ir.IRModule()
    mod = mod.from_expr(x)

    return mod, params

## Create and show the model

In [3]:
mod, params = create_model()
print(mod)

Loaded random array from "weights.npy"
def @main(%input: Tensor[(1, 3, 16, 16), int8], %conv1.weights: Tensor[(32, 3, 3, 3), int8], %conv1.bias: Tensor[(32), int32]) {
  %0 = qnn.conv2d(%input, %conv1.weights, 0, 0, 1f, 1f, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3], out_dtype="int32");
  %1 = nn.bias_add(%0, %conv1.bias);
  %2 = right_shift(%1, 4);
  %3 = clip(%2, a_min=-128f, a_max=127f);
  cast(%3, dtype="int8")
}



## Make it a TVMModel

In [4]:
model = TVMCModel(mod, params)

# Compile the model to C code

## Compilation options

In [5]:
target = "soma_dory, c"      # send supported operations to the digital accelerator, generate C code for the CPU for all other operations
#target = "c"                # generate C code for the CPU only

fuse_layers = True           # enable/disable layer fusion

## Compile the model

Compile the TVM model and unpack the generated .tar file to a given build folder (build_path).

The generated source code can be found in `build_folder/codegen/host/src`.

Uitleg over output

In [6]:
tvmc_compile_and_unpack(model, target=target, fuse_layers=fuse_layers, build_path='build')


Backend: Matching patterns from generated DORY ONNX to HW Nodes.

Find One other solution, It will not work for real networks with multiple strides = 2

Diana Backend: Adjusting Data Layout to HWC and CoutKCin.

Updating memory occupation and MACs of tensors in layers

Insert tiling parameters per layer inside graph nodes

DORY Backend: Renaming Weights tensors.

Mapping the layers files to their templates and copying the kernels associated.

Generating weight string for tvmgen_default_soma_dory_main_0.
def @main(%input: Tensor[(1, 3, 16, 16), int8] /* ty=Tensor[(1, 3, 16, 16), int8] */) -> Tensor[(1, 32, 16, 16), int8] {
  %0 = reshape(%input, newshape=[192, 4]) /* ty=Tensor[(192, 4), int8] */;
  %1 = reverse(%0, axis=1) /* ty=Tensor[(192, 4), int8] */;
  %2 = reshape(%1, newshape=[1, 3, 16, 16]) /* ty=Tensor[(1, 3, 16, 16), int8] */;
  %3 = @tvmgen_default_soma_dory_main_0(%2) /* ty=Tensor[(1, 32, 16, 16), int8] */;
  %4 = reshape(%3, newshape=[2048, 4]) /* ty=Tensor[(2048, 4), int8

[09:25:30] /esat/sol1/users/jvandelm/gitlab-runner/builds/soma_compiler/tvm-fork/src/relay/backend/aot_executor_codegen.cc:497: CreateFuncCall: tvmgen_default_fused_reshape_reverse_reshape -> tir.tvm_check_return(0, -1, tir.call_extern("tvmgen_default_fused_reshape_reverse_reshape", input_buffer_var, sid_1))

[09:25:30] /esat/sol1/users/jvandelm/gitlab-runner/builds/soma_compiler/tvm-fork/src/relay/backend/aot_executor_codegen.cc:497: CreateFuncCall: tvmgen_default_soma_dory_main_0 -> tir.tvm_check_return(0, -1, tir.call_extern("tvmgen_default_soma_dory_main_0", sid_1, sid_2))

[09:25:30] /esat/sol1/users/jvandelm/gitlab-runner/builds/soma_compiler/tvm-fork/src/relay/backend/aot_executor_codegen.cc:497: CreateFuncCall: tvmgen_default_fused_reshape_reverse_reshape_1 -> tir.tvm_check_return(0, -1, tir.call_extern("tvmgen_default_fused_reshape_reverse_reshape_1", sid_2, output_buffer_var))



# Compile generated C code for DIANA
## Copy the DORY runtime library files

The generated model code makes calls to the DORY runtime library, which contains the microkernel implementation of all supported operations for the digital accelerator. We need to include these files into the build. A clone of DORY is installed in `/dory`

In [7]:
DORY_SRC_DIR="/dory"
DORY_DST_DIR="dory"

In [8]:
!mkdir -p $DORY_DST_DIR/include
!mkdir -p $DORY_DST_DIR/src

!cp $DORY_SRC_DIR/dory/Hardware_targets/Diana/Backend_Kernels/dory-hal/include/*.h $DORY_DST_DIR/include
!cp $DORY_SRC_DIR/dory/Hardware_targets/Diana/Backend_Kernels/dory-hal/src/*.c $DORY_DST_DIR/src
!cp $DORY_SRC_DIR/dory/Hardware_targets/Diana/Diana_TVM/Utils_files/*.h $DORY_DST_DIR/include
!cp $DORY_SRC_DIR/dory/Hardware_targets/Diana/Diana_TVM/Utils_files/*.c $DORY_DST_DIR/src

## Generate a template application

For this, we:
* copy a few C files (including a malloc wrapper, prolfiling tools and some required header files)
* Generate a main template function

In [9]:
APP_DST_DIR="app"
APP_SRC_DIR="../../byoc"

In [10]:
!mkdir -p $APP_DST_DIR/src
!mkdir -p $APP_DST_DIR/include

!cp $APP_SRC_DIR/src/*.c $APP_DST_DIR/src
!cp $APP_SRC_DIR/include/*.h $APP_DST_DIR/include

In [11]:
create_demo_file(mod, APP_DST_DIR + '/src/demo.c') # generate demo.c, the template main function

Creating demo file: Inferring shapes and types...
Creating demo file: Inferred shapes:
	input (int8):
	 [1 3 16 16]
	output (int8):
	 [1 32 16 16]


## Cross-compile for DIANA

Open a terminal and execute `make -f Makefile.pulprt clean all`